In [1]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
import numpy as np
from scipy.optimize import minimize

def merged_hotelling_profits(phi, lambd, merged_idxs=(0,1), wtp=None, mc=None):
    # If you want mc flexible, make sure you pass it in from the rest of your code!
    n = 3
    phi = np.array(phi)
    if mc is None:
        mc = np.zeros(n)
    else:
        mc = np.array(mc)
    a, b = merged_idxs
    c = [i for i in range(n) if i not in merged_idxs][0]

    def shares(prices):
        p = prices
        s = np.zeros(3)
        z_02 = 1/6 + (p[2]-p[0])/(2*lambd)
        z_10 = 1/6 + (p[0]-p[1])/(2*lambd)
        z_21 = 1/6 + (p[1]-p[2])/(2*lambd)
        s[0] = z_02 + (1/6 - (p[0]-p[1])/(2*lambd))
        s[1] = z_21 + (1/6 - (p[1]-p[0])/(2*lambd))
        s[2] = 1 - s[0] - s[1]
        return s

    def merged_profit(pr, p_c):
        prices = np.zeros(3)
        prices[a], prices[b], prices[c] = pr[0], pr[1], p_c
        s = shares(prices)
        return -((prices[a]-phi[a])*s[a] + (prices[b]-phi[b])*s[b])

    def rival_profit(p_c, pr):
        prices = np.zeros(3)
        prices[a], prices[b], prices[c] = pr[0], pr[1], p_c
        s = shares(prices)
        return -(prices[c]-phi[c])*s[c]

    p_init = np.ones(3) * (np.mean(phi) + lambd/3)
    pr = [p_init[a], p_init[b]]
    p_other = p_init[c]

    for _ in range(100):
        pr_old = np.array(pr)
        p_other_old = p_other
        # merged firm best response
        res_m = minimize(merged_profit, pr, args=(p_other,), method='Nelder-Mead')
        pr = res_m.x
        # rival response
        res_c = minimize(rival_profit, [p_other], args=(pr,), method='Nelder-Mead')
        p_other = res_c.x[0]
        if np.max(np.abs(np.array(pr) - pr_old)) < 1e-8 and abs(p_other - p_other_old) < 1e-8:
            break

    prices = np.zeros(3)
    prices[a], prices[b], prices[c] = pr[0], pr[1], p_other
    s = shares(prices)
    hospital_profit = np.sum(s * (phi - mc))  # uses actual mc
    insurer_profits = s * (prices - phi)
    return hospital_profit, insurer_profits, s, prices

# Example usage:
phi = np.array([15, 15, 15])
lambd = 5
wtp = 25
hospital_profit, insurer_profits, shares, prices = merged_hotelling_profits(phi, lambd)
print("hospital profit:", hospital_profit)
print("insurer profits:", insurer_profits)
print("shares:", shares)
print("prices:", prices)


#TODO passive disageement needs a firm specific index
#TODO probalby need a firm specific index for the merger too...

hospital profit: 0.0
insurer profits: [1.04111194e+308             inf             inf]
shares: [ 2.65783269e+154  1.12395971e+153 -2.77022866e+154]
prices: [ 3.91714627e+153  2.80940012e+155 -7.32245085e+153]


C:\Users\ericsc\AppData\Local\Temp\ipykernel_4236\4131290967.py:30: RuntimeWarning: overflow encountered in double_scalars
  return -((prices[a]-phi[a])*s[a] + (prices[b]-phi[b])*s[b])
C:\Users\ericsc\AppData\Local\anaconda3\lib\site-packages\scipy\optimize\_optimize.py:863: RuntimeWarning: invalid value encountered in subtract
  np.max(np.abs(fsim[0] - fsim[1:])) <= fatol):
C:\Users\ericsc\AppData\Local\Temp\ipykernel_4236\4131290967.py:36: RuntimeWarning: overflow encountered in double_scalars
  return -(prices[c]-phi[c])*s[c]
C:\Users\ericsc\AppData\Local\Temp\ipykernel_4236\4131290967.py:58: RuntimeWarning: overflow encountered in multiply
  insurer_profits = s * (prices - phi)


In [3]:



def solve_hotelling3(phi, lambd):
    n = 3
    A = np.eye(n)
    for i in range(n):
        A[i, (i-1)%n] -= 0.5
        A[i, (i+1)%n] -= 0.5
    b = np.zeros(n)
    for i in range(n):
        b[i] = phi[i] - 0.5*(phi[(i-1)%n] + phi[(i+1)%n]) + lambd
    A_full = np.vstack([A, np.ones(n)])
    b_full = np.append(b, np.mean(phi) + lambd/3)
    price = np.linalg.lstsq(A_full, b_full, rcond=None)[0]
    return price


def solve_hotelling2(phi, lambd, idx):
    i, j = idx
    p_i = (2 * phi[i] + phi[j]) / 3 + lambd
    p_j = (2 * phi[j] + phi[i]) / 3 + lambd
    prices = np.zeros(len(phi))
    prices[i] = p_i
    prices[j] = p_j
    return prices


def calc_price(phi, lambd, wtp):
    phi = np.array(phi)
    n = len(phi)
    price = np.zeros(n)
    eps = 1e-8

    if n == 1:
        price = np.array([max(phi[0], min(wtp, wtp-lambd))])
    elif n == 2:
        price = solve_hotelling2(phi, lambd, [0, 1])
    elif n == 3:
        # All equal? Symmetric solution
        if np.max(phi) - np.min(phi) < eps:
            price = np.ones(n) * (phi[0] + lambd/3)
        else:
            # Try general solution
            price_try = solve_hotelling3(phi, lambd)
            ok = (price_try >= phi - eps) & (price_try <= wtp + eps)
            if np.all(ok):
                price = price_try
            else:
                # Find two lowest-cost firms, use 2-firm solution
                active = np.argsort(phi)[:2]
                price = solve_hotelling2(phi, lambd, active)
    else:
        raise NotImplementedError("Only supports n=1, 2, or 3.")
    return price

    
def shares_hotelling3(prices, cost):
    n = len(prices)
    s = np.zeros(n)
    circumference = 1.0
    for i in range(n):
        left = (i-1) % n
        right = (i+1) % n
        arc = circumference / n
        delta_p_left = prices[i] - prices[left]
        z_left = (delta_p_left / (2*cost)) + (arc/2)
        delta_p_right = prices[right] - prices[i]
        z_right = (delta_p_right / (2*cost)) + (arc/2)
        s[i] = z_left + z_right
    s *= (circumference / np.sum(s))  # normalize
    return s    


def shares_hotelling2(prices, cost, idx):
    # idx: indices of two active firms
    i, j = idx
    p_i, p_j = prices[i], prices[j]
    s_i = 0.5 + (p_j - p_i) / (2 * cost)
    s_j = 1.0 - s_i
    s = np.zeros(len(prices))
    s[i], s[j] = s_i, s_j
    return s
    
    
def calc_s(phi, cost, wtp, circle=True):
    phi = np.array(phi)
    n = len(phi)
    prices = calc_price(phi, cost, wtp)

    if np.all(prices == 0):
        return np.zeros(n)

    active = np.where(prices > 0)[0]

    if len(active) == 1:
        s = np.zeros(n)
        s[active[0]] = 1.0
        return s
    elif len(active) == 2:
        return shares_hotelling2(prices, cost, active)
    elif len(active) == 3:
        # All active; circle
        s = shares_hotelling3(prices, cost)
        # If crazy numbers (e.g., shares < 0), fallback to zeros.
        if np.any(s < 0) or np.any(s > 1):
            return np.zeros(n)
        else:
            return s
    else:
        return np.zeros(n)


def calc_profits(phi, cost, wtp, mc):
    s = calc_s(phi, cost, wtp)
    prices = calc_price(phi, cost, wtp)
    hospital_profit = np.sum(s * (phi - mc))
    insurer_profits = s * (prices - phi)
    return hospital_profit, insurer_profits, s, prices


def analytic_phi(cost, wtp, n, belief="active"):
    if n==0:
        return 0
    if n == 1:
        return (wtp - cost) / 2
    elif n == 2:
        if belief == "active":
            return 0.75*cost + (wtp-cost)/2
        elif belief == "passive":
            return 1.5*cost
    elif n == 3:
        if belief == "active":
            return cost + wtp/2
        elif belief == "passive":
            return 9/4*cost
    return None


def nash_obj_helper(phi_vec, cost, wtp, mc, betas, disagreement=0, merged_idxs=None, k_active=0, lambd=5):
    # merger skip logic: skip duplicate merged agent
    if merged_idxs is not None and k_active in merged_idxs and merged_idxs.index(k_active) == 1:
        return 1e8

    if merged_idxs is not None:
        hosp, ins, s, prices = merged_hotelling_profits(phi_vec, lambd, merged_idxs, wtp, mc)
    else:
        hosp, ins, s, prices = calc_profits(phi_vec, cost, wtp, mc)
    if hosp - disagreement <= 1e-8 or ins[0] <= 1e-8 or hosp <= 1e-8 :
        return 10
    obj = np.log(hosp - disagreement) * (1 - betas[0]) + np.log(ins[0]) * betas[0]
    return -obj


#TODO nash objective actually depends on the thing we're optimzing over e.g. dphi1/dphi2 = 0... 
def passive_disagreement(phi_vec, cost, wtp, mc, merged_idxs=None):
    s = calc_s(phi_vec, cost, wtp)
    phi = np.array(phi_vec)
    mc = np.array(mc)

    if merged_idxs is None:
        return np.sum(s * (phi - mc)) - s[0] * (phi[0] - mc[0])

    # For merger: hospital gets only business from non-merged outlets
    merged_idxs = list(merged_idxs)
    other_idxs = [i for i in range(len(phi)) if i not in merged_idxs]
    return np.sum([s[i] * (phi[i] - mc[i]) for i in other_idxs])

def recursive_disagreement(phi_vec, cost, wtp, mc, betas, analytic_base=True):
    n = len(phi_vec)
    if analytic_base and n <= 3:
        # For 1- or 2-firm base: use all analytic phi (active beliefs by default here)
        sub_phi = np.array([analytic_phi(cost, wtp, n-1, belief='active')] * n)
        hospital_profit, _, _, _ = calc_profits(sub_phi, cost, wtp, mc[:n])
        return hospital_profit
    if n == 1:
        # (should never reach here if analytic_base is True... but as fallback)
        phi_init = [analytic_phi(cost, wtp, 1, belief='active')]
        result = minimize(lambda phi: nash_obj(phi, cost, wtp, [mc[0]], [betas[0]]),
                          phi_init, method='Nelder-Mead', options={'disp': False})
        phi_star = result.x
        hosp_profit, _, _, _ = calc_profits(phi_star, cost, wtp, [mc[0]])
        return hosp_profit
    # n-1 agents: recursively solve Nash for subgame (all but the first)
    keep = list(range(1, n))
    phi_sub = np.array([phi_vec[j] for j in keep])
    mc_sub = np.array([mc[j] for j in keep])
    betas_sub = [betas[j] for j in keep]
    phi_sub_sol = solve_nash(phi_sub, cost, wtp, mc_sub, betas_sub, analytic_base=analytic_base)
    hospital_profit, _, _, _ = calc_profits(phi_sub_sol, cost, wtp, mc_sub)
    return hospital_profit


def nash_obj(
        phi_vec, cost, wtp, mc, betas,
        active=True, analytic_base=True, phi_tmp_prev=0,
        merged_idxs=None, k_active=0, lambd=5):

    n = len(phi_vec)
    if n == 1:
        return 0
    if active:
        # You may want to update recursive_disagreement for merger logic as well if needed
        disagreement = recursive_disagreement(phi_vec, cost, wtp, mc, betas, analytic_base=analytic_base)
    else:
        phi_vec_tmp = phi_vec.copy()
        phi_vec_tmp[0] = phi_tmp_prev  # keep the current firm phi fixed...
        disagreement = passive_disagreement(phi_vec_tmp, cost, wtp, mc, merged_idxs)
    return nash_obj_helper(phi_vec, cost, wtp, mc, betas, disagreement, merged_idxs, k_active, lambd=lambd)


def solve_nash(
        phi_init, cost, wtp, mc, betas=None, merged_idxs=None, lambd=5,
        active=False, maxiter=100, tol=1e-7, analytic_base=True, verbose=False):
    n = len(phi_init)
    if betas is None:
        betas = [0.5] * n
    phi = np.array(phi_init, dtype=float)
    phi_prev = phi.copy()

    for it in range(maxiter):
        phi_prev[:] = phi[:]
        for k in range(n):
            def helper(phi_k_scalar):
                phi_tmp = phi_prev.copy()
                phi_tmp_prev = phi_tmp[0]
                phi_tmp[k] = phi_k_scalar[0]
                phi_tmp_roll = np.roll(phi_tmp, -k)
                mc_roll = np.roll(mc, -k)
                betas_roll = np.roll(betas, -k)
                return nash_obj(phi_tmp_roll, cost, wtp, mc_roll, betas_roll,
                                active=active, analytic_base=analytic_base, phi_tmp_prev=phi_tmp_prev,
                                merged_idxs=merged_idxs, k_active=0, lambd=lambd)
            res = minimize(helper, [phi[k]], method='Nelder-Mead', options={'disp': False})
            phi[k] = res.x[0]
        diff = np.max(np.abs(phi - phi_prev))
        phi_prev = phi.copy()
        if verbose:
            print(f"Iter {it}, phi={phi}, diff={diff}")
        if diff < tol:
            break
    return phi

In [4]:
phi = np.array([14.4375,13.75,13.75])
lambd = 5.0  # try as needed
wtp = 25.0  # just needs to be larger than all phi+lambd
prices = calc_price(phi, lambd, wtp)
print("Prices:", prices)

Prices: [ 0.   18.75 18.75]


In [5]:
calc_price([ 0. ,  18.75 ,18.75], lambd, wtp)

array([11.25, 17.5 ,  0.  ])

In [6]:
lambd = 5.0  # try as needed
wtp = 25.0  # just needs to be larger than all phi+lambd
s = calc_s([14.4375,13.75,13.75], lambd, wtp)
print("shares:", s )

shares: [0.  0.5 0.5]


In [7]:
# --- Model run/prints ---
COST = 6
WTP = 25
MC = np.array([0, 0, 0])
betas = [0.5, 0.5, 0.5]
betas2 = [0.5, 0.5]

n =3
phi_init_pass = [analytic_phi(COST, WTP, n, belief='passive')]*n
phi_passive = solve_nash(phi_init_pass, COST, WTP, np.zeros(n), [0.5]*n, active=False)
print("PASSIVE φ:", phi_passive)
hospital_profit, insurer_profits, s, prices = calc_profits(phi_passive, COST, WTP, np.zeros(n))
print("   hospital profit:", hospital_profit, "insurers:", insurer_profits, "shares:", s, "prices:", prices)
print("Analytic φ (passive):", analytic_phi(COST, WTP, n, "passive"))

PASSIVE φ: [13.48353046 13.48353046 13.48353046]
   hospital profit: 13.483530461543836 insurers: [0.66666667 0.66666667 0.66666667] shares: [0.33333333 0.33333333 0.33333333] prices: [15.48353046 15.48353046 15.48353046]
Analytic φ (passive): 13.5


In [8]:
# --- Model run/prints ---
COST = 5
WTP = 26

MC = np.array([0, 0, 0])
betas = [0.5, 0.5, 0.5]
betas2 = [0.5, 0.5]

n =3
phi_init_act = [analytic_phi(COST, WTP, n, belief='active')]*n
phi_active = solve_nash(phi_init_act, COST, WTP, np.zeros(n), [0.5]*n, active=True)
print("ACTIVE φ:", phi_active)
hospital_profit_a, insurer_profits_a, s_a, prices_a = calc_profits(phi_active, COST, WTP, np.zeros(n))
print("   hospital profit:", hospital_profit_a, "insurers:", insurer_profits_a, "shares:", s_a, "prices:", prices_a)
print("Analytic φ (active):", analytic_phi(COST, WTP, n, "active"))

ACTIVE φ: [17.98901699 17.98901699 17.98901699]
   hospital profit: 17.989016990447013 insurers: [0.55555556 0.55555556 0.55555556] shares: [0.33333333 0.33333333 0.33333333] prices: [19.65568366 19.65568366 19.65568366]
Analytic φ (active): 18.0
